# BLIP-2 on CrisisMMD v1.0 (Part B)

This Colab notebook:
1. **Downloads a prepared CrisisMMD subset** (images + `subset.csv` with columns: `image_path,type,danger,action,split,image_id`)
2. **Fine-tunes BLIP-2 with LoRA** (Q-Former only)
3. **Generates** the deliverable `generated_reports.csv` (columns: `Image_ID,Generated_Report,Danger_Level`)

⚠️ **Prereqs**: Provide a downloadable URL (`.zip`) containing:
- A folder `images/` with all images used by the subset
- A CSV `subset.csv` with at least the columns: `image_path,type,danger,action,split,image_id`
- `image_path` values should be relative paths like `images/xxx.jpg`


In [2]:
# 0. Environment setup (no bitsandbytes for CPU compatibility)
!pip -q install -U transformers accelerate datasets peft pillow torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

import os, sys, json, re, csv
import torch
from PIL import Image
from datasets import load_dataset
from transformers import Blip2Processor, Blip2ForConditionalGeneration, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)


ERROR: Could not find a version that satisfies the requirement dill<0.3.9,>=0.3.0 (from datasets) (from versions: none)
ERROR: No matching distribution found for dill<0.3.9,>=0.3.0


PackageNotFoundError: dill

In [ ]:
if use_lora:
    peft_cfg = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["query", "key", "value", "dense"]  # BERT-style
    )
    model = get_peft_model(model, peft_cfg)

In [ ]:
#@title 2. Prepare dataset objects
import pandas as pd
CSV_PATH = os.path.join(SAVE_DIR, 'subset.csv')
df = pd.read_csv(CSV_PATH)
required_cols = {'image_path','type','danger','action','split','image_id'}
assert required_cols.issubset(set(df.columns)), f'Missing required columns: {required_cols - set(df.columns)}'

def make_target(row):
    return f"Type: {row['type']} | Danger: {row['danger']} | Action: {row['action']}"

df['target'] = df.apply(make_target, axis=1)
train_df = df[df['split']=='train'].copy()
val_df   = df[df['split']=='val'].copy()
test_df  = df[df['split']=='test'].copy()

train_csv = os.path.join(SAVE_DIR,'train.csv')
val_csv   = os.path.join(SAVE_DIR,'val.csv')
test_csv  = os.path.join(SAVE_DIR,'test.csv')
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)
test_df.to_csv(test_csv, index=False)

dataset = load_dataset('csv', data_files={'train':train_csv,'validation':val_csv})
print(dataset)

In [ ]:
#@title 3. Processor and preprocess function
MODEL_NAME = 'Salesforce/blip2-flan-t5-xl'  # switch to xx-large if VRAM allows
PROMPT = "Generate a disaster danger report in the format: Type: <TYPE> | Danger: <High/Medium/Low> | Action: <ACTION>"
processor = Blip2Processor.from_pretrained(MODEL_NAME)

IMG_ROOT = SAVE_DIR  # image_path is relative to this root

from PIL import Image
def preprocess(ex):
    path = os.path.join(IMG_ROOT, ex['image_path'])
    image = Image.open(path).convert('RGB')
    inputs = processor(images=image, text=PROMPT, return_tensors='pt')
    labels = processor.tokenizer(ex['target'], return_tensors='pt').input_ids
    ex['pixel_values'] = inputs['pixel_values'][0]
    ex['input_ids'] = inputs['input_ids'][0]
    ex['attention_mask'] = inputs['attention_mask'][0]
    ex['labels'] = labels[0]
    return ex

remove_cols = list(set(df.columns) - {'image_id'}) + ['target']
dataset = dataset.map(preprocess, remove_columns=remove_cols, num_proc=1)
dataset

In [ ]:
#@title 4. Load model and enable LoRA on Q-Former
from transformers import Blip2ForConditionalGeneration
from peft import LoraConfig, get_peft_model, TaskType

model = Blip2ForConditionalGeneration.from_pretrained(MODEL_NAME)

# Freeze everything
for n, p in model.named_parameters():
    p.requires_grad = False

# Unfreeze Q-Former and language projection
for n, p in model.named_parameters():
    if n.startswith('qformer') or 'language_projection' in n:
        p.requires_grad = True

# Apply LoRA to Q-Former attention layers
peft_cfg = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=[
        'qformer.encoder.layer.0.attention.self.query',
        'qformer.encoder.layer.0.attention.self.key',
        'qformer.encoder.layer.0.attention.self.value',
    ]
)
model = get_peft_model(model, peft_cfg)
model.to(device)
model.print_trainable_parameters()

In [ ]:
#@title 5. Training
from transformers import TrainingArguments, Trainer
import torch

args = TrainingArguments(
    output_dir='/content/blip2_crisismmd_qformer',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    fp16=True,
    report_to='none'
)

def collate(batch):
    keys = ['pixel_values','input_ids','attention_mask','labels']
    out = {}
    for k in keys:
        if k == 'pixel_values':
            out[k] = torch.stack([b[k] for b in batch])
        else:
            out[k] = torch.nn.utils.rnn.pad_sequence([b[k] for b in batch], batch_first=True)
    return out

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    data_collator=collate,
)
trainer.train()

trainer.save_model('/content/blip2_crisismmd_qformer')
processor.save_pretrained('/content/blip2_crisismmd_qformer')

In [ ]:
#@title 6. Inference on test set and CSV export
import pandas as pd, re
ckpt = '/content/blip2_crisismmd_qformer'
processor = Blip2Processor.from_pretrained(ckpt)
from transformers import Blip2ForConditionalGeneration
model = Blip2ForConditionalGeneration.from_pretrained(ckpt).to(device).eval()

def normalize(report:str):
    m = re.search(r"Type:\s*([^|]+)\|\s*Danger:\s*([^|]+)\|\s*Action:\s*(.+)$", report)
    if not m:
        return "Type: Unknown | Danger: Medium | Action: Monitoring."
    t, d, a = [x.strip() for x in m.groups()]
    d = d.capitalize()
    if d not in ["High","Medium","Low"]:
        d = "Medium"
    return f"Type: {t} | Danger: {d} | Action: {a}"

test_df = pd.read_csv(os.path.join(SAVE_DIR,'test.csv'))
rows = []
for _, r in test_df.iterrows():
    img_path = os.path.join(IMG_ROOT, r['image_path'])
    image = Image.open(img_path).convert('RGB')
    inputs = processor(images=image, text=PROMPT, return_tensors='pt')
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        out = model.generate(**inputs, num_beams=4, max_new_tokens=64, length_penalty=0.1, no_repeat_ngram_size=3)
    text = processor.tokenizer.decode(out[0], skip_special_tokens=True)
    rep = normalize(text)
    m = re.search(r"Danger:\s*(High|Medium|Low)", rep)
    danger = m.group(1) if m else 'Medium'
    rows.append([r['image_id'], rep, danger])

out_csv = '/content/generated_reports.csv'
pd.DataFrame(rows, columns=['Image_ID','Generated_Report','Danger_Level']).to_csv(out_csv, index=False)
print('Saved ->', out_csv)

### Notes
- Ensure your subset zip contains `subset.csv` and an `images/` folder.
- The model fine-tunes **Q-Former** with **LoRA**, keeping the vision encoder and LLM frozen.
- Output strictly matches exam deliverable: `generated_reports.csv`.